In [8]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Load the video file
cap = cv2.VideoCapture("Танец - Made with Clipchamp.mp4")

# Create a pose estimator
pose = mp.solutions.pose.Pose()

In [9]:
bone_name = [
    "nose", 
    "left_eye_inner",
    "left_eye",
    "left_eye_outer",
    "right_eye_inner",
    "right_eye",
    "right_eye_outer",
    "left_ear", 
    "right_ear", 
    "mouth_left", 
    "mouth_right", 
    "left_shoulder", 
    "right_shoulder", 
    "left_elbow", 
    "right_elbow", 
    "left_wrist", 
    "right_wrist", 
    "left_pinky", 
    "right_pinky", 
    "left_index", 
    "right_index", 
    "left_thumb", 
    "right_thumb", 
    "left_hip", 
    "right_hip", 
    "left_knee", 
    "right_knee", 
    "left_ankle", 
    "right_ankle",
    "left_heel",
    "right_heel",
    "left_foot_index", 
    "right_foot_index"
    ]

In [10]:
poses = []
offset = []
i = 0
# Start the video capture loop
while True:
	# Get the current frame from the video
	ret, frame = cap.read()
	if not ret:
		break
	cap.set(3, 640) # Width
	cap.set(4, 480) # Lenght
	cap.set(10, 100) # Brightness



	# Estimate the pose of the person in the current frame
	results = pose.process(frame)
	

	landmarks = np.zeros((len(results.pose_landmarks.landmark), 3))
	for j, landmark in enumerate(results.pose_landmarks.landmark):
		landmarks[j] = [landmark.x, -landmark.z, -landmark.y]
	landmarks = [landmarks]
	landmarks = np.array(landmarks)
	if len(offset) == 0:
		center = np.mean(landmarks, axis=(0, 1))

		offset = -center

	landmarks += offset

	for landmark in landmarks:
		# Draw the pose of the person on the current frame
		if results.pose_landmarks:
			frame_pose = {
				"frame": i,
				"pose": {}
				}
			for j in range(len(bone_name)):
				frame_pose["pose"][bone_name[j]] = {
					"x": landmark[j][0],
					"y": landmark[j][1],
					"z": landmark[j][2],
					}
				#cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)
			poses.append(frame_pose)

	# Display the current frame
	#cv2.imshow("Frame", frame)
	i+=1

	# Check if the user wants to quit
	if cv2.waitKey(1) & 0xFF == 27:
		break

# Close the video capture object
cap.release()

# Destroy all the windows
cv2.destroyAllWindows()

In [ ]:
import json
#poses = moving_average(poses, 15)
with open("poses.json", "w") as f:
	json.dump(poses, f)